## Quantum Dynamical Model for a Frenkel Exciton 
[Jay Foley and Lane Tolley, University of North Carolina Charlotte](https://foleylab.github.io/)

#### objectives
-Implement function that will build the Hamiltonian for a Frenkel Exciton within a 2D film

#### Summary
The time evolution of energy transferred within a 2D crystalline organic film can be described with excitonic degrees of freedom; the purpose of this notebook is to implement a function that will build the hamiltonian for such a system.
    

## Import libraries

In [1]:
import numpy as np
import scipy
import math

# Will come back and create a vizualization 

## Theory

The time evolution of energy transfer within a crystalline 2D film of a few organic monomers can be described with excitonic degrees of freedom. 

Quantum dynamical systems can be described with the time dependent Schrodinger equation (TDSE):

$$ i\hbar \frac{d}{dt}\Psi(x,t) = \hat{H} \Psi(x,t) \tag{1} $$

The Hamiltonian $\hat{H}$ of this system is comprised of three changeable parameters: the number of basis sets, the exciton energy, and interparticle coupling. The number of basis sets will change based upon the number of monomers included within the system, the exciton energy is a value that will be approximated and chosen for the system, and the coupling term will require the use of another function, $\hat{V}_{nm}$:

$$ V_{nm} = \frac{1}{{n^2}{r^3}} \left( \mu_n\cdot \mu_m - 3\frac{(\mu_n \cdot {\bf r})({\bf r} \cdot \mu_m)}{r^2}\right) \tag{2} $$
where $\mu_n$ is the transition dipole moment associated with the exciton on site $n$, ${\bf r}$ denotes the separation vector between site $n$ and $m$, and $r$ denotes the scalar separation between $n$ and $m$. 


In [2]:
def compute_dd_coupling(mu_d, mu_a, r_vector, n):
    """ Function that computes the dipole dipole coupling contribution of the total energy of a system based upon mu_d,
        the transition dipole moment of the donor, mu_a, the transition dipole moment of the acceptor, r_vector, the 
        distance separating the donor and acceptor, and the refractive index, a paremeter that describes the effect of 
        the system on light.
    
    Arguments
    ---------
    mu_d : numpy array of floats
        the transition dipole mooment of the donor
    mu_a : numpy array of floats
        the transition dipole moment of the acceptor
    r : numpy array of floats
        the distance separating the donor and acceptor
    n : float
        refractive index of the medium
    """

    r_scalar = np.sqrt(np.dot(r_vector, r_vector))
    
    return (1 / (n ** 2 * r_scalar ** 3)) * (np.dot(mu_d, mu_a) - 3 * (np.dot(mu_d, r_vector) * np.dot(r_vector, mu_a)) / r_scalar ** 2)

mu_d1 = np.array([0,0,1]) 
mu_a1 = np.array([0,0,1])
r_1 = [1,0,0] 
n_1 = 1.0

dd_1 = compute_dd_coupling(mu_d1, mu_a1, r_1, n_1)
print(dd_1)

# only change x and z designation for mu_a and mu_d (2D)
    

1.0


## The Hamiltonian
The Hamiltonian of this system can be tentatively described by eqation 3:

$$ \hat{H} = \hat{H}_0 + \hat{V}. \tag{3} $$

Using the notation from [Willard and co-workers](https://pubs.acs.org/doi/10.1021/acs.jpcc.8b11504)  

\begin{equation}
\hat{H}_0 = \sum_n \epsilon_n |n \rangle \langle n | 
\end{equation}
and 
\begin{equation}
\hat{V} = \sum_{nm} V_{nm} |n\rangle \langle m|
\end{equation}

Here the projection operators will lead to diagonal contributions for $\hat{H}_0$ 
\begin{align}
\langle p | \hat{H}_0 | q \rangle &= \langle p | \sum_n \epsilon_n |n \rangle \langle n |  q \rangle \\
&= \langle p | \epsilon_q |q \rangle \langle q |  q \rangle \\ 
& = \langle p | \epsilon_q |q \rangle \cdot 1 \\
&= \epsilon_q \langle p | q \rangle \\
&= \epsilon_q \delta_{pq}    
\end{align}
and off-diagonal contributions for $\hat{V}$:
\begin{align}
\langle p | \hat{V} | q \rangle &= \langle p | \sum_{nm} V_{nm} |n \rangle \langle m |  q \rangle \\
&= \langle p | \sum_{n} V_{nq} |n \rangle \langle q |  q \rangle \\ 
& = \langle p | \sum_{n} V_{nq} |n \rangle \cdot 1 \\
& =  \sum_{n} V_{nq} \langle p | n \rangle  \\
&= \sum_{n} V_{nq}\delta_{pn}  \\ 
&= V_{pq}
\end{align}
where we note that an exciton does not interact with itself through this potential, so elements $V_{qq}$ are necessarily zero.

In [15]:
def H0_matrix_element(n, m, epsilon_m):
    """ Helper function to take two indices (n, m) and return the H_0 matrix element H_{n,m}, 
        which is <n|H_0|m> in Dirac's Bra-Ket notation.
    
    Arguments
    ---------
    n : int
        the index corresponding to the bra state
    m : int
        the index corresponding to the ket state 
        
    epsilon_m : float
        the exciton energy of site m
    
    Returns
    -------
    H_nm : float 
        Hamiltonian matrix element <n|H_0 | m> = \epsilon_m <n|m>
    
    """
    H_nm = (1 + dd_1) * (n == m)
    return H_nm

def build_hamiltonian(n_basis, exciton_energy, V_AD):
    """ Function that builds the Hamailtonian which models the time evolution of an excitonic system based upon the
        field free energy of the system and the dipole dipole coupling of the sysetem
        
    Arguments
    ---------
    H_o : numpy array of floats
        a numpy array of spectral data fed to the function
    V_AD : Float
        dipole dipole coupling term that is given by the compute_dd_coupling() functionFunction that builds the Hamailtonian which models the time evolution of an excitonic system based upon the
        field free energy of the system and the dipole dipole coupling of the sysetem
        
    """
    
    H = np.zeros((n_basis, n_basis))
    
    for i in range(n_basis):
        n = i 
        for j in range(n_basis):
            m = j 
            H[i, j] = H_matrix_element(n, m)
    return H

build_hamiltonian(2, 1, dd_1)

# Do I need to build a function that will parse a spectral data file and turn it into a matrix for exciton_energy?


array([[2., 0.],
       [0., 2.]])